In [8]:
%cd ..

/workspaces/mlops-student-performance


/home/codespace/.local/share/virtualenvs/mlops-student-performance-Zsm4y_dL/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [13]:
import mlflow
from mlflow import MlflowClient

import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

from src.web_service.models.features import Features

In [2]:
EXPERIMENT = 'student_performance'
TRACKING_SERVER_HOST = "localhost"
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")
mlflow.set_experiment(EXPERIMENT)
MLFLOW_TRACKING_URI = mlflow.get_tracking_uri()

In [3]:
client = MlflowClient(MLFLOW_TRACKING_URI)

model_name = "student-performance-predictor"
model_alias = 'best'

client.get_model_version_by_alias(model_name, model_alias)

<ModelVersion: aliases=['best'], creation_timestamp=1728359853673, current_stage='None', description='', last_updated_timestamp=1728359853673, name='student-performance-predictor', run_id='f120b50c04ea4edf9d37ecef1e53e243', run_link='', source='mlflow-artifacts:/1/f120b50c04ea4edf9d37ecef1e53e243/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='5'>

In [4]:
model_version_info = client.get_model_version_by_alias(model_name, model_alias)
model_version = model_version_info.version
RUN_ID = model_version_info.run_id


model_uri = f"models:/{model_name}/{model_version}"
model = mlflow.pyfunc.load_model(model_uri)

/home/codespace/.local/share/virtualenvs/mlops-student-performance-Zsm4y_dL/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

2024/10/08 13:35:07 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - numpy (current: 2.1.2, required: numpy==1.26.4)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


In [5]:
RUN_ID

'f120b50c04ea4edf9d37ecef1e53e243'

In [29]:
features = {
    "Hours_Studied": 10.5,
    "Attendance": 95,
    "Parental_Involvement": "High",
    "Access_to_Resources": "Medium",
    "Extracurricular_Activities": "Yes",
    "Sleep_Hours": 7.5,
    "Previous_Scores": 85.5,
    "Motivation_Level": "Medium",
    "Internet_Access": "Yes",
    "Tutoring_Sessions": 2,
    "Family_Income": "Medium",
    "Teacher_Quality": "High",
    "School_Type": "Public",
    "Peer_Influence": "Positive",
    "Physical_Activity": 3.5,
    "Learning_Disabilities": "No",
    "Parental_Education_Level": "College",
    "Distance_from_Home": "Moderate",
    "Gender": "Female"
}
input_data = pd.DataFrame([features])
input_data


,Hours_Studied,Attendance,Parental_Involvement,Access_to_Resources,Extracurricular_Activities,Sleep_Hours,Previous_Scores,Motivation_Level,Internet_Access,Tutoring_Sessions,Family_Income,Teacher_Quality,School_Type,Peer_Influence,Physical_Activity,Learning_Disabilities,Parental_Education_Level,Distance_from_Home,Gender
0,10.5,95,High,Medium,Yes,7.5,85.5,Medium,Yes,2,Medium,High,Public,Positive,3.5,No,College,Moderate,Female


In [36]:
cat_cols = input_data.select_dtypes(include=['object', 'category']).columns
ord_encoder = OrdinalEncoder()
input_data.loc[:, cat_cols] = ord_encoder.fit_transform(input_data[cat_cols])
input_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Hours_Studied               1 non-null      float64
 1   Attendance                  1 non-null      int64  
 2   Parental_Involvement        1 non-null      object 
 3   Access_to_Resources         1 non-null      object 
 4   Extracurricular_Activities  1 non-null      object 
 5   Sleep_Hours                 1 non-null      float64
 6   Previous_Scores             1 non-null      float64
 7   Motivation_Level            1 non-null      object 
 8   Internet_Access             1 non-null      object 
 9   Tutoring_Sessions           1 non-null      int64  
 10  Family_Income               1 non-null      object 
 11  Teacher_Quality             1 non-null      object 
 12  School_Type                 1 non-null      object 
 13  Peer_Influence              1 non-null 

In [34]:
preds = model.predict(input_data)

#float(preds[0])

MlflowException: Failed to enforce schema of data '   Hours_Studied  Attendance Parental_Involvement Access_to_Resources  \
0           10.5          95                  0.0                 0.0   

  Extracurricular_Activities  Sleep_Hours  Previous_Scores Motivation_Level  \
0                        0.0          7.5             85.5              0.0   

  Internet_Access  Tutoring_Sessions Family_Income Teacher_Quality  \
0             0.0                  2           0.0             0.0   

  School_Type Peer_Influence  Physical_Activity Learning_Disabilities  \
0         0.0            0.0                3.5                   0.0   

  Parental_Education_Level Distance_from_Home Gender  
0                      0.0                0.0    0.0  ' with schema '['Hours_Studied': double (required), 'Attendance': double (required), 'Parental_Involvement': double (required), 'Access_to_Resources': double (required), 'Extracurricular_Activities': double (required), 'Sleep_Hours': double (requ